In [1]:
!pip install langchain-community faiss-cpu

  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -------------------- ------------------- 1.3/2.5 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 9.1 MB/s  0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 12.6 MB/s  0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 15.0 MB/s  0:00:00
   ---------------------------------------- 0.0/18.8 MB ? eta -:--:--
   ----- ---------------------------------- 2.6/18.8 MB 12.5 MB/s eta 0:00:02
   ---

In [2]:
from typing import TypedDict, List
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END

# --- 1. SETUP MODELS (LOCAL) ---
# We use Qwen for chatting and Nomic (or Qwen) for embeddings.
# Run 'ollama pull nomic-embed-text' for better results, or just use qwen.
llm = ChatOllama(model="qwen2.5:0.5b", temperature=0)
embeddings = OllamaEmbeddings(model="qwen2.5:0.5b") 

# --- 2. PREPARE DOCUMENTS (The "Knowledge Base") ---
# In a real app, you would load these from PDFs.
docs = [
    Document(
        page_content=(
            "LangGraph is a powerful library used for building stateful and multi-actor "
            "applications that run on top of large language models (LLMs). It allows developers "
            "to design workflows where multiple steps or agents communicate with each other. "
            "LangGraph makes it easy to create chatbots, automation pipelines, retrieval systems, "
            "and AI agents that maintain memory and state across interactions. It is commonly used "
            "for building complex AI applications with simple, modular blocks."
        ),
        metadata={"source": "doc1"}
    ),

    Document(
        page_content=(
            "Himanshu Yadav is currently a B.Tech student at IIT Kanpur and is expected to graduate "
            "in the year 2026. He has strong interests in software development, machine learning, "
            "and AI-based automation. During his time at IIT Kanpur, he has worked on several "
            "projects related to data science, operating systems, and AI agents. Himanshu is also "
            "focused on improving communication skills and preparing for interviews to pursue a tech "
            "career in startups and product-based companies."
        ),
        metadata={"source": "doc2"}
    ),

    Document(
        page_content=(
            "The weather in Kanpur is currently hot and humid, which is typical during the summer "
            "season in northern India. Temperatures usually rise above 40°C, and humidity increases "
            "due to moisture in the air. This weather often leads to warm nights and requires people "
            "to stay hydrated and avoid going out during peak afternoon heat. Local residents rely "
            "heavily on coolers and air conditioners during this period."
        ),
        metadata={"source": "doc3"}
    ),

    Document(
        page_content=(
            "Ollama is a simple and efficient tool that allows users to run open-source language models "
            "locally on their computer. With Ollama, you can download models like Llama, Mistral, "
            "and Gemma and run them without needing cloud services. It is very useful for developers "
            "who want full control over their models, offline usage, lower latency, or better privacy. "
            "Ollama also provides an easy command-line interface for managing and interacting with "
            "different AI models."
        ),
        metadata={"source": "doc4"}
    ),
]


print("Indexing documents... (This might take a moment locally)")
vector_store = FAISS.from_documents(docs, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 2}) # Retrieve top 2 matches

# --- 3. DEFINE STATE ---
class RAGState(TypedDict):
    question: str
    context: List[Document]
    answer: str

# --- 4. DEFINE NODES ---

def retrieve_node(state: RAGState):
    """
    Step 1: Find relevant documents based on the question.
    """
    print(f"🔍 Searching for: '{state['question']}'")
    retrieved_docs = retriever.invoke(state["question"])
    return {"context": retrieved_docs}

def generate_node(state: RAGState):
    """
    Step 2: Generate an answer using the retrieved context.
    """
    question = state["question"]
    docs = state["context"]
    
    # Combine content from the retrieved docs
    context_text = "\n\n".join([d.page_content for d in docs])
    
    prompt = ChatPromptTemplate.from_template(
        """You are a helpful assistant. Answer the question ONLY based on the context provided below.
        If the answer is not in the context, say "I don't know".
        
        Context:
        {context}
        
        Question: {question}
        Answer:"""
    )
    
    chain = prompt | llm
    response = chain.invoke({"context": context_text, "question": question})
    
    return {"answer": response.content}

# --- 5. BUILD GRAPH ---

workflow = StateGraph(RAGState)

workflow.add_node("retrieve", retrieve_node)
workflow.add_node("generate", generate_node)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

app = workflow.compile()

# --- 6. RUN IT ---

# Test 1: A question about the docs
print("\n--- TEST 1 ---")
inputs = {"question": "Who is Himanshu?"}
result = app.invoke(inputs)
print(f"🤖 Answer: {result['answer']}")

# Test 2: Another question about the docs
print("\n--- TEST 2 ---")
inputs = {"question": "What is LangGraph used for?"}
result = app.invoke(inputs)
print(f"🤖 Answer: {result['answer']}")

# Test 3: A question NOT in the docs (Hallucination check)
print("\n--- TEST 3 ---")
inputs = {"question": "What is the capital of France?"}
result = app.invoke(inputs)
print(f"🤖 Answer: {result['answer']}")

Indexing documents... (This might take a moment locally)

--- TEST 1 ---
🔍 Searching for: 'Who is Himanshu?'
🤖 Answer: Himanshu Yadav is a B.Tech student at IIT Kanpur who is expected to graduate in the year 2026 and has strong interests in software development, machine learning, and AI-based automation. He has worked on several projects related to data science, operating systems, and AI agents during his time at IIT Kanpur. Himanshu is also focused on improving communication skills and preparing for interviews to pursue a tech career in startups and product-based companies.

--- TEST 2 ---
🔍 Searching for: 'What is LangGraph used for?'
🤖 Answer: LangGraph is a powerful library used for building stateful and multi-actor applications that run on top of large language models (LLMs). It allows developers to design workflows where multiple steps or agents communicate with each other.

--- TEST 3 ---
🔍 Searching for: 'What is the capital of France?'
🤖 Answer: I don't know.
